In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import time

In [2]:
df = pd.read_parquet('C:/Users/Ticy/Desktop/移動平均放入後所有單年度/移動平均放入後2022.parquet', engine='pyarrow')
df = df.drop(['year', 'date'], axis=1)
print(df.shape)

(5100018, 11)


In [3]:
df.head()

,month,day,hour,weekday,PULocationID,is_holiday,次數,Nhp,7days_moving_average,Name_lyft,Name_uber
17349905,1,1,21,5,135,1,14,uber_21_135,16,0,1
17349898,1,1,21,5,128,1,2,uber_21_128,3,0,1
17349899,1,1,21,5,129,1,158,uber_21_129,134,0,1
17349884,1,1,21,5,114,1,182,uber_21_114,129,0,1
17349885,1,1,21,5,115,1,4,uber_21_115,8,0,1


In [4]:
valid = pd.read_parquet('C:/Users/Ticy/Desktop/移動平均放入後所有單年度/移動平均放入後2023.parquet', engine='pyarrow')
valid = valid.drop(['year', 'date'], axis=1)

In [5]:
train = df[~df['month'].isin([11, 12])]
test = df[df['month'].isin([11, 12])]
valid = valid[valid['month'] == 1]

In [6]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
train['Nhp'] = label.fit_transform(train['Nhp'])
test['Nhp'] = label.fit_transform(test['Nhp'])
valid['Nhp'] = label.fit_transform(valid['Nhp'])

C:\Users\Ticy\AppData\Local\Temp\ipykernel_11108\3645517485.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Nhp'] = label.fit_transform(train['Nhp'])
C:\Users\Ticy\AppData\Local\Temp\ipykernel_11108\3645517485.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Nhp'] = label.fit_transform(test['Nhp'])


In [7]:
train = train.rename(columns={'次數': 'count'})
test = test.rename(columns={'次數': 'count'})
valid = valid.rename(columns={'次數': 'count'})

In [8]:
X_train = train.drop('count', axis=1).values
y_train = train['count'].values
X_test = test.drop('count', axis=1).values
y_test = test['count'].values
X_valid = valid.drop('count', axis=1).values
y_valid = valid['count'].values

## scikit-learn

In [9]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


scaler = StandardScaler()
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

## LightGBM

In [10]:
start_time = time.time()


from sklearn.model_selection import GridSearchCV
kf = KFold(n_splits=5, shuffle=True, random_state=0)
param_grid = {'n_estimators': range(100, 501, 100),
              'learning_rate': np.logspace(0, -2, 3), 
              'max_depth': range(3, 11)}

lgbm = LGBMRegressor(n_jobs=-1, 
                     random_state=0, 
                     verbose=-1)

model_cv = GridSearchCV(lgbm, param_grid, cv=kf, scoring='r2')
model_cv.fit(X_train_scaled, y_train)
print(model_cv.best_params_, model_cv.best_score_)


end_time = time.time()
execution_time = end_time - start_time
print("執行時間:", execution_time, "秒")

KeyboardInterrupt: 

In [17]:
start_time = time.time()


model = LGBMRegressor(n_estimators=100, 
                      learning_rate=1, 
                      max_depth=8, 
                      num_leaves=31,
                      n_jobs=-1, 
                      random_state=0, 
                      verbose=-1)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_valid_scaled)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}\nR2: {r2}')


end_time = time.time()
execution_time = end_time - start_time
print("執行時間:", execution_time, "秒")

MAE: 15.107376763178433
R2: 0.868309928829479
執行時間: 6.936546564102173 秒


In [21]:
y_actual

array([ 1, 12, 49, ...,  5, 11, 41], dtype=int64)

In [23]:
prediction

array([68.23359422, 68.23359422, 68.23359422, ..., 56.91757214,
       56.91757214, 56.91757214])

In [18]:
prediction = model.predict(X_actual)
mean_absolute_error(y_actual, prediction)
r2 = r2_score(y_actual, prediction)
print(f'MAE: {mae}\nR2: {r2}')

MAE: 15.107376763178433
R2: -0.04944763525821494


## NN

In [ ]:
# start_time = time.time()


# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Input
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import BatchNormalization
# from tensorflow.keras.layers import LeakyReLU

# n_cols = X_train.shape[1]

# model = Sequential([Dense(14, input_shape=(n_cols,)),
#                     BatchNormalization(),
#                     LeakyReLU(),
#                     Dense(14, input_shape=(n_cols,)),
#                     BatchNormalization(),
#                     LeakyReLU(),
#                     Dense(1)
# ])

# model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['accuracy'])
# model.summary()

# # 擬合X, y
# history = model.fit(X_train, y_train, validation_split=0.3, batch_size=100, epochs=10, verbose=2)


# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.legend(['training', 'validation'], loc='upper left')
# plt.show()


# result = model.evaluate(X_test, y_test)
# print(result)


# end_time = time.time()
# execution_time = end_time - start_time
# print("執行時間:", execution_time, "秒")